Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Logistic - Part 1
---

In [10]:
l2_list = np.logspace(-4, -2, num=8)
l2_list

array([ 0.0001    ,  0.00019307,  0.00037276,  0.00071969,  0.0013895 ,
        0.0026827 ,  0.00517947,  0.01      ])

In [11]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
RELU_nodes = 1024


    
graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels ],seed=13))
    biases = tf.Variable(tf.zeros([num_labels]))
    
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*tf.nn.l2_loss(weights) ) 

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  tf.matmul(tf_valid_dataset, weights) + biases
    logits_test =  tf.matmul(tf_test_dataset, weights) + biases
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)

In [12]:
num_steps = 3001


for l2_pen in l2_list:
    print('-------------------------------------------------------')
    print('lambda penlty: ',l2_pen)
    tf.reset_default_graph()
    with tf.Session(graph=graph) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
        tf.global_variables_initializer().run()
        print('Initialized')
        for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, 
                         tf_train_labels : batch_labels,
                         lam_1 : l2_pen}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

-------------------------------------------------------
lambda penlty:  0.0001
Initialized
Minibatch loss at step 0: 17.395771
Minibatch accuracy: 8.6%
Validation accuracy: 13.0%
Minibatch loss at step 500: 1.686778
Minibatch accuracy: 76.6%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.656845
Minibatch accuracy: 75.0%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 1.439626
Minibatch accuracy: 72.7%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 1.239918
Minibatch accuracy: 79.7%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 1.233304
Minibatch accuracy: 78.9%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 1.095397
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Test accuracy: 87.1%
-------------------------------------------------------
lambda penlty:  0.000193069772888
Initialized
Minibatch loss at step 0: 17.677162
Minibatch accuracy: 8.6%
Validation accuracy: 13.0%
Minibatch loss at step 500: 1.858493
Minibatch accuracy: 76.6%
Va

---
NN - Part 2
---

In [13]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
RELU_nodes = 1024

def hidden_layer1(dataset,weights1,weights2,biases1,biases2):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    logits = tf.matmul(layer_1, weights2) + biases2
    return logits

    
graph_nn = tf.Graph()
with graph_nn.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, RELU_nodes ],seed=13))
    biases1 = tf.Variable(tf.zeros([RELU_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([RELU_nodes, num_labels],seed=713))
    biases2 = tf.Variable(tf.zeros([num_labels]))  
  
    
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = hidden_layer1(tf_train_dataset,weights1,weights2,biases1,biases2)
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) ) ) 

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_layer1(tf_valid_dataset,weights1,weights2,biases1,biases2)
    logits_test =  hidden_layer1(tf_test_dataset,weights1,weights2,biases1,biases2)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)

In [14]:
num_steps = 3001


for l2_pen in l2_list:
    print('-------------------------------------------------------')
    print('lambda penlty: ',l2_pen)
    tf.reset_default_graph()
    with tf.Session(graph=graph_nn) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
        tf.global_variables_initializer().run()
        print('Initialized')
        for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, 
                         tf_train_labels : batch_labels,
                         lam_1 : l2_pen}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

-------------------------------------------------------
lambda penlty:  0.0001
Initialized
Minibatch loss at step 0: 301.306610
Minibatch accuracy: 5.5%
Validation accuracy: 37.4%
Minibatch loss at step 500: 44.861843
Minibatch accuracy: 78.1%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 39.046978
Minibatch accuracy: 76.6%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 31.301304
Minibatch accuracy: 72.7%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 32.691940
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 27.343849
Minibatch accuracy: 82.8%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 26.430088
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Test accuracy: 86.4%
-------------------------------------------------------
lambda penlty:  0.000193069772888
Initialized
Minibatch loss at step 0: 330.556641
Minibatch accuracy: 5.5%
Validation accuracy: 37.4%
Minibatch loss at step 500: 65.107269
Minibatch accuracy:

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [40]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
RELU_nodes = 1024

def hidden_layer1(dataset,weights1,weights2,biases1,biases2):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    logits = tf.matmul(layer_1, weights2) + biases2
    return logits

    
graph_OF = tf.Graph()
with graph_OF.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, RELU_nodes ],seed=13))
    biases1 = tf.Variable(tf.zeros([RELU_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([RELU_nodes, num_labels],seed=713))
    biases2 = tf.Variable(tf.zeros([num_labels]))  
  
    
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = hidden_layer1(tf_train_dataset,weights1,weights2,biases1,biases2)
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) ) )
   

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_layer1(tf_valid_dataset,weights1,weights2,biases1,biases2)
    logits_test =  hidden_layer1(tf_test_dataset,weights1,weights2,biases1,biases2)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)

In [42]:
num_steps = 3001

l2_pen =0.0014
tf.reset_default_graph()
with tf.Session(graph=graph_OF) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        ## limit the training data to only 5 batches
        offset = batch_size*np.random.choice(5)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, 
                tf_train_labels : batch_labels,
                lam_1 : l2_pen}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 692.769531
Minibatch accuracy: 14.1%
Validation accuracy: 32.1%
Minibatch loss at step 500: 218.410202
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 108.422195
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 53.826050
Minibatch accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 2000: 26.730869
Minibatch accuracy: 99.2%
Validation accuracy: 75.8%
Minibatch loss at step 2500: 13.270087
Minibatch accuracy: 100.0%
Validation accuracy: 76.1%
Minibatch loss at step 3000: 6.597422
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Test accuracy: 84.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [45]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
RELU_nodes = 1024

def hl1_dropout(dataset,weights1,weights2,biases1,biases2):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    layer1_dropout = tf.nn.dropout(layer_1,0.50)
    logits = tf.matmul(layer1_dropout, weights2) + biases2
    return logits

def hidden_layer1(dataset,weights1,weights2,biases1,biases2):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    logits = tf.matmul(layer_1, weights2) + biases2
    return logits

    
graph_OF = tf.Graph()
with graph_OF.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, RELU_nodes ],seed=13))
    biases1 = tf.Variable(tf.zeros([RELU_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([RELU_nodes, num_labels],seed=713))    layer1_dropout = tf.nn.dropout(layer_1,0.50)
    biases2 = tf.Variable(tf.zeros([num_labels]))  
  
    
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = hl1_dropout(tf_train_dataset,weights1,weights2,biases1,biases2)
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) ) )
   

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_layer1(tf_valid_dataset,weights1,weights2,biases1,biases2)
    logits_test =  hidden_layer1(tf_test_dataset,weights1,weights2,biases1,biases2)
    logits = hidden_layer1(tf_train_dataset,weights1,weights2,biases1,biases2)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 3001

l2_pen =0.0014
tf.reset_default_graph()
with tf.Session(graph=graph_OF) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        ## limit the training data to only 5 batches
        offset = batch_size*np.random.choice(5)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, 
                tf_train_labels : batch_labels,
                lam_1 : l2_pen}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 869.677734
Minibatch accuracy: 7.8%
Validation accuracy: 30.1%
Minibatch loss at step 500: 221.705292
Minibatch accuracy: 99.2%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 109.532623
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 54.389408
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 27.007586
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 13.417537
Minibatch accuracy: 99.2%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 6.669506
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Test accuracy: 86.5%


# Despite the training data being severly overfit (the minibatch accuracy was 100%)  the Test and Validation error are less when using the dropout method


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## Just Playing around below to try to increase Test Accuracy but best was still with a single Relu in part 1 at 93.2%

In [81]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 96
RELU_nodes = 256


def h2_drop(dataset,weights1,weights2,weights3,biases1,biases2,biases3):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.matmul(layer_1,weights2) + biases2
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,0.50)
    logits = tf.matmul(layer_2, weights3) + biases3
    return logits

def hidden_2layer(dataset,weights1,weights2,weights3,biases1,biases2,biases3):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.matmul(layer_1,weights2) + biases2
    layer_2 = tf.nn.relu(layer_2)
    logits = tf.matmul(layer_2, weights3) + biases3
    return logits

    
graph_OF = tf.Graph()
with graph_OF.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, RELU_nodes ], stddev=0.1,seed=13))
    biases1 = tf.Variable(tf.zeros([RELU_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([RELU_nodes, RELU_nodes], stddev=0.1))
    biases2 = tf.Variable(tf.zeros([RELU_nodes]))  
    
    weights3 = tf.Variable(
        tf.truncated_normal([RELU_nodes, num_labels], stddev=0.1))
    biases3 = tf.Variable(tf.zeros([num_labels])) 
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.009, global_step,1000, 0.6, staircase=True)
    #print('learning_rate=', learning_rate)
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    #logits = hl2_dropout(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    #logits = hidden_2layer(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    logits = h2_drop(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*(tf.nn.l2_loss(weights1) + 
                     tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3)) )
   

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_2layer(tf_valid_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    logits_test =  hidden_2layer(tf_test_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    logits = hidden_2layer(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 6001
l2_pen = 0.007
tf.reset_default_graph()
with tf.Session(graph=graph_OF) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        ## limit the training data to only 5 batches
        #offset = batch_size*np.random.choice(5)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, 
                tf_train_labels : batch_labels,
                lam_1 : l2_pen}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 10.856944
Minibatch accuracy: 2.1%
Validation accuracy: 5.2%
Minibatch loss at step 500: 7.527697
Minibatch accuracy: 85.4%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 7.423432
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 6.731667
Minibatch accuracy: 89.6%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 6.797565
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 6.482297
Minibatch accuracy: 80.2%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 6.538231
Minibatch accuracy: 77.1%
Validation accuracy: 82.2%
Minibatch loss at step 3500: 6.410162
Minibatch accuracy: 79.2%
Validation accuracy: 82.2%
Minibatch loss at step 4000: 6.291559
Minibatch accuracy: 83.3%
Validation accuracy: 82.3%
Minibatch loss at step 4500: 6.273638
Minibatch accuracy: 84.4%
Validation accuracy: 82.3%
Minibatch loss at step 5000: 6.144545
Minibatch accuracy: 84.4%
Validation accuracy

In [65]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 96
RELU_nodes = 256

def h5_drop(dataset,weights,biases):
    layer_1 = tf.matmul(dataset, weights['hl1']) + biases['b1']
    layer_1 = tf.nn.relu(layer_1)
    #layer_1 = tf.nn.dropout(layer_1,0.5)
    layer_2 = tf.matmul(layer_1 , weights['hl2']) + biases['b2']
    layer_2 = tf.nn.relu(layer_2)
    #layer_2 = tf.nn.dropout(layer_2,0.5)
    layer_3 = tf.matmul(layer_2 , weights['hl3']) + biases['b3']
    layer_3 = tf.nn.relu(layer_3)
    #layer_3 = tf.nn.dropout(layer_3,0.50)
    layer_4 = tf.matmul(layer_3 , weights['hl4']) + biases['b4']
    layer_4 = tf.nn.relu(layer_4)
    layer_4 = tf.nn.dropout(layer_4,0.5)
    logits = tf.matmul(layer_4,  weights['hl5']) + biases['b5']
    return logits

def hidden_5layer(dataset,weights,biases):
    layer_1 = tf.matmul(dataset, weights['hl1']) + biases['b1']
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.matmul(layer_1 , weights['hl2']) + biases['b2']
    layer_2 = tf.nn.relu(layer_2)
    
    layer_3 = tf.matmul(layer_2 , weights['hl3']) + biases['b3']
    layer_3 = tf.nn.relu(layer_3)
    
    layer_4 = tf.matmul(layer_3 , weights['hl4']) + biases['b4']
    layer_4 = tf.nn.relu(layer_4)
    
    logits = tf.matmul(layer_4,  weights['hl5']) + biases['b5']
    return logits

    
graph_OF = tf.Graph()
with graph_OF.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    
    # Create a dict for weights & biases to pass and call in function
    weights = {
        'hl1': tf.Variable(tf.truncated_normal(
                    [image_size * image_size, RELU_nodes ], stddev=0.2,seed=13)),
        'hl2': tf.Variable(tf.truncated_normal(
                    [RELU_nodes, RELU_nodes ], stddev=0.1)),
        'hl3': tf.Variable(tf.truncated_normal(
                    [RELU_nodes, RELU_nodes ], stddev=0.2)),
        'hl4': tf.Variable(tf.truncated_normal(
                    [RELU_nodes, RELU_nodes ], stddev=0.1)),
        'hl5': tf.Variable(tf.truncated_normal(
                    [RELU_nodes, num_labels ], stddev=0.2))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([RELU_nodes])),
        'b2': tf.Variable(tf.zeros([RELU_nodes])),
        'b3': tf.Variable(tf.zeros([RELU_nodes])),
        'b4': tf.Variable(tf.zeros([RELU_nodes])),
        'b5': tf.Variable(tf.zeros([num_labels])),
    }

    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.0004, global_step,1000, 0.65, staircase=True)
    #print('learning_rate=', learning_rate)
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = h5_drop(tf_train_dataset,weights,biases)
    #logits = hidden_5layer(tf_train_dataset,weights,biases)
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*(tf.nn.l2_loss(weights['hl1']) + 
                     tf.nn.l2_loss(weights['hl2']) + 
                     tf.nn.l2_loss(weights['hl3']) +
                     tf.nn.l2_loss(weights['hl4']) +
                     tf.nn.l2_loss(weights['hl5'])) )
   

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_5layer(tf_valid_dataset,weights,biases)
    logits_test =  hidden_5layer(tf_test_dataset,weights,biases)
    logits = hidden_5layer(tf_train_dataset,weights,biases)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 7001
l2_pen = 0.5
tf.reset_default_graph()
with tf.Session(graph=graph_OF) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        ## limit the training data to only 5 batches
        #offset = batch_size*np.random.choice(5)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, 
                tf_train_labels : batch_labels,
                lam_1 : l2_pen}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2356.952393
Minibatch accuracy: 4.2%
Validation accuracy: 7.3%
Minibatch loss at step 500: 1909.985229
Minibatch accuracy: 63.5%
Validation accuracy: 59.5%
Minibatch loss at step 1000: 1563.508911
Minibatch accuracy: 66.7%
Validation accuracy: 67.0%
Minibatch loss at step 1500: 1372.771973
Minibatch accuracy: 66.7%
Validation accuracy: 69.3%
Minibatch loss at step 2000: 1205.840332
Minibatch accuracy: 66.7%
Validation accuracy: 71.0%
Minibatch loss at step 2500: 1108.193481
Minibatch accuracy: 77.1%
Validation accuracy: 72.1%
Minibatch loss at step 3000: 1018.715149
Minibatch accuracy: 67.7%
Validation accuracy: 72.8%
Minibatch loss at step 3500: 964.320984
Minibatch accuracy: 69.8%
Validation accuracy: 73.1%
Minibatch loss at step 4000: 912.987732
Minibatch accuracy: 70.8%
Validation accuracy: 73.4%
Minibatch loss at step 4500: 880.916809
Minibatch accuracy: 77.1%
Validation accuracy: 73.6%
Minibatch loss at step 5000: 850.127441
Minibatch accurac

In [89]:
#With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
RELU_nodes = 256


def h2_drop(dataset,weights1,weights2,weights3,biases1,biases2,biases3):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.matmul(layer_1,weights2) + biases2
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,0.50)
    logits = tf.matmul(layer_2, weights3) + biases3
    return logits

def hidden_2layer(dataset,weights1,weights2,weights3,biases1,biases2,biases3):
    layer_1 = tf.matmul(dataset, weights1) + biases1
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.matmul(layer_1,weights2) + biases2
    layer_2 = tf.nn.relu(layer_2)
    logits = tf.matmul(layer_2, weights3) + biases3
    return logits

    
graph_OF = tf.Graph()
with graph_OF.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
#Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lam_1 = tf.placeholder(tf.float32)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, RELU_nodes ], stddev=0.1,seed=13))
    biases1 = tf.Variable(tf.zeros([RELU_nodes]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([RELU_nodes, RELU_nodes], stddev=0.1))
    biases2 = tf.Variable(tf.zeros([RELU_nodes]))  
    
    weights3 = tf.Variable(
        tf.truncated_normal([RELU_nodes, num_labels], stddev=0.1))
    biases3 = tf.Variable(tf.zeros([num_labels])) 
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.01, global_step,1000, 0.55, staircase=True)
    #print('learning_rate=', learning_rate)
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = hidden_2layer(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    #logits = h2_drop(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) 
            + lam_1*(tf.nn.l2_loss(weights1) + 
                     tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3)) )
   

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    logits_val =  hidden_2layer(tf_valid_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    logits_test =  hidden_2layer(tf_test_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    logits = hidden_2layer(tf_train_dataset,weights1,weights2,weights3,biases1,biases2,biases3)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_val)
    test_prediction = tf.nn.softmax(logits_test)
    
num_steps = 6001
l2_pen = 0.003
tf.reset_default_graph()
with tf.Session(graph=graph_OF) as session:
# This is a one-time operation which ensures the parameters get initialized as
# we described in the graph: random weights for the matrix, zeros for the
# biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        ## limit the training data to only 5 batches
        #offset = batch_size*np.random.choice(5)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, 
                tf_train_labels : batch_labels,
                lam_1 : l2_pen}
        _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.906355
Minibatch accuracy: 11.7%
Validation accuracy: 12.5%
Minibatch loss at step 500: 3.680832
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 3.654895
Minibatch accuracy: 80.5%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 3.556173
Minibatch accuracy: 82.8%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 3.519262
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 3.405211
Minibatch accuracy: 82.8%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 3.392290
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 3500: 3.263624
Minibatch accuracy: 89.1%
Validation accuracy: 82.4%
Minibatch loss at step 4000: 3.397538
Minibatch accuracy: 83.6%
Validation accuracy: 82.4%
Minibatch loss at step 4500: 3.293275
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 5000: 3.315751
Minibatch accuracy: 82.8%
Validation accurac